In [1]:
library(tidyverse)
library(repr)
library(tidymodels)


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.6     ✔ rsample      1.2.1
✔ dials        1.3.0     ✔ tune         1.1.2
✔ infer        1.0.7     ✔ workflows    1.1.4
✔ modeldata    1.4.0     ✔ workflowsets 1.0.1
✔ parsnip      1.2.1     ✔ yardstick    1.3.1
✔ recipes      1.1.0     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

In [5]:
# reading in data
players_data<- read_csv("players.csv") |>
               mutate(gender = as_factor(gender))

Rows: 196 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, age
lgl (3): subscribe, individualId, organizationName

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [ ]:
## set.seed(9456)

options(repr.plot.width=10, repr.plot.height=6)

## filtering people with zero , selected wanted parameters
players <- filter(players_data, played_hours != 0) |> 
                         select(played_hours, age, gender) |>
                         arrange(desc(age))

## creating the train/test split

players_split <- initial_split(players, prop = 0.75, strata = gender)
players_train <- training(players_split)
players_test <- testing(players_split)

## Preprocessing the data

players_recipe <- recipe(gender ~ played_hours + gender, data = players_train) |>
    step_scale(all_predictors()) |>
    step_center(all_predictors())

## Trainig classifier

knn_spec <- nearest_neighbor(weight_func = "rectangular" , 
                             neighbors = 5) |> 
     set_engine("kknn") |>
     set_mode("classification")

knn_fit <- workflow() |> 
           add_recipe(players_recipe) |>
           add_model(knn_spec) |>
           fit(players_train)

## making predictions

gender_predicted <- predict(knn_fit, players_train) |>
                    bind_cols(players_train)
        
# compute the accuracy
acc <- gender_predicted |>
  metrics(truth = gender, estimate = .pred_class) |>
  filter(.metric == "accuracy") |>
  select(.estimate) |>
  pull()
acc
